## Question_5_Solution:

In [49]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime

In [95]:
import requests
import json

response_API = requests.get('http://api.tvmaze.com/singlesearch/shows?q=westworld&embed=episodes')
data = response_API.text

# Parse JSON data
parse_json = json.loads(data)

# Remove the first object
if "_embedded" in parse_json:
    filtered_data = parse_json["_embedded"].get("episodes")
    if filtered_data and len(filtered_data) > 0:
        filtered_data.pop(0)
        
print(filtered_data)

[{'id': 911201, 'url': 'https://www.tvmaze.com/episodes/911201/westworld-1x02-chestnut', 'name': 'Chestnut', 'season': 1, 'number': 2, 'type': 'regular', 'airdate': '2016-10-09', 'airtime': '21:00', 'airstamp': '2016-10-10T01:00:00+00:00', 'runtime': 60, 'rating': {'average': 7.7}, 'image': {'medium': 'https://static.tvmaze.com/uploads/images/medium_landscape/78/195436.jpg', 'original': 'https://static.tvmaze.com/uploads/images/original_untouched/78/195436.jpg'}, 'summary': '<p>Bernard suspects that someone is sabotaging the hosts. Meanwhile, the Man continues his search for the deeper game, and Maeve finds herself in a world of hurt... literally.</p>', '_links': {'self': {'href': 'https://api.tvmaze.com/episodes/911201'}, 'show': {'href': 'https://api.tvmaze.com/shows/1371'}}}, {'id': 911204, 'url': 'https://www.tvmaze.com/episodes/911204/westworld-1x03-the-stray', 'name': 'The Stray', 'season': 1, 'number': 3, 'type': 'regular', 'airdate': '2016-10-16', 'airtime': '21:00', 'airstamp'

In [96]:
# Checking filtered dataframe:
df = pd.json_normalize(filtered_data)
df.head()

,id,url,name,season,number,type,airdate,airtime,airstamp,runtime,summary,rating.average,image.medium,image.original,_links.self.href,_links.show.href
0,911201,https://www.tvmaze.com/episodes/911201/westwor...,Chestnut,1,2,regular,2016-10-09,21:00,2016-10-10T01:00:00+00:00,60,<p>Bernard suspects that someone is sabotaging...,7.7,https://static.tvmaze.com/uploads/images/mediu...,https://static.tvmaze.com/uploads/images/origi...,https://api.tvmaze.com/episodes/911201,https://api.tvmaze.com/shows/1371
1,911204,https://www.tvmaze.com/episodes/911204/westwor...,The Stray,1,3,regular,2016-10-16,21:00,2016-10-17T01:00:00+00:00,60,<p>Bernard continues to investigate Dolores' s...,7.6,https://static.tvmaze.com/uploads/images/mediu...,https://static.tvmaze.com/uploads/images/origi...,https://api.tvmaze.com/episodes/911204,https://api.tvmaze.com/shows/1371
2,911205,https://www.tvmaze.com/episodes/911205/westwor...,Dissonance Theory,1,4,regular,2016-10-23,21:00,2016-10-24T01:00:00+00:00,60,<p>While Dolores joins William and Logan on th...,7.9,https://static.tvmaze.com/uploads/images/mediu...,https://static.tvmaze.com/uploads/images/origi...,https://api.tvmaze.com/episodes/911205,https://api.tvmaze.com/shows/1371
3,927174,https://www.tvmaze.com/episodes/927174/westwor...,Contrapasso,1,5,regular,2016-10-30,21:00,2016-10-31T01:00:00+00:00,60,<p>Dolores takes the first step on her path of...,8.0,https://static.tvmaze.com/uploads/images/mediu...,https://static.tvmaze.com/uploads/images/origi...,https://api.tvmaze.com/episodes/927174,https://api.tvmaze.com/shows/1371
4,932057,https://www.tvmaze.com/episodes/932057/westwor...,The Adversary,1,6,regular,2016-11-06,21:00,2016-11-07T02:00:00+00:00,60,"<p>While Maeve convinces Felix to show her ""up...",8.0,https://static.tvmaze.com/uploads/images/mediu...,https://static.tvmaze.com/uploads/images/origi...,https://api.tvmaze.com/episodes/932057,https://api.tvmaze.com/shows/1371


In [88]:
df.columns

Index(['id', 'url', 'name', 'season', 'number', 'type', 'airdate', 'airtime',
       'airstamp', 'runtime', 'summary', 'rating.average', 'image.medium',
       'image.original', '_links.self.href', '_links.show.href'],
      dtype='object')

In [103]:
# function to get air time in 12 hour format:
df['airtime']= pd.to_datetime(df['airtime'], format='%H:%M').dt.strftime('%I:%M %p')
df['airtime']

ValueError: unconverted data remains when parsing with format "%H:%M": " PM", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [100]:
# Extracing string by remove HTML tags from summary column:
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

# Apply the function to the column:
df['summary'] = df['summary'].apply(remove_html_tags)

In [104]:
# Rename columns
df = df.rename(columns={'rating.average': 'average rating', 'image.medium': 'medium image link', 'image.original': 'Original image link'})

In [105]:
required_columns = ['id', 'url', 'name', 'season', 'number', 'type', 'airdate', 'airtime', 'runtime', 'average rating', 'summary', 'medium image link', 'Original image link']

In [106]:
# final dataframe:
required_df = df[required_columns]
required_df.head()

,id,url,name,season,number,type,airdate,airtime,runtime,average rating,summary,medium image link,Original image link
0,911201,https://www.tvmaze.com/episodes/911201/westwor...,Chestnut,1,2,regular,2016-10-09,09:00 PM,60,7.7,Bernard suspects that someone is sabotaging th...,https://static.tvmaze.com/uploads/images/mediu...,https://static.tvmaze.com/uploads/images/origi...
1,911204,https://www.tvmaze.com/episodes/911204/westwor...,The Stray,1,3,regular,2016-10-16,09:00 PM,60,7.6,Bernard continues to investigate Dolores' supp...,https://static.tvmaze.com/uploads/images/mediu...,https://static.tvmaze.com/uploads/images/origi...
2,911205,https://www.tvmaze.com/episodes/911205/westwor...,Dissonance Theory,1,4,regular,2016-10-23,09:00 PM,60,7.9,While Dolores joins William and Logan on their...,https://static.tvmaze.com/uploads/images/mediu...,https://static.tvmaze.com/uploads/images/origi...
3,927174,https://www.tvmaze.com/episodes/927174/westwor...,Contrapasso,1,5,regular,2016-10-30,09:00 PM,60,8.0,Dolores takes the first step on her path of di...,https://static.tvmaze.com/uploads/images/mediu...,https://static.tvmaze.com/uploads/images/origi...
4,932057,https://www.tvmaze.com/episodes/932057/westwor...,The Adversary,1,6,regular,2016-11-06,09:00 PM,60,8.0,"While Maeve convinces Felix to show her ""upsta...",https://static.tvmaze.com/uploads/images/mediu...,https://static.tvmaze.com/uploads/images/origi...


In [108]:
# saving to CSV file:
required_df.to_csv('Question_5_Solution.csv', index=False)